In [ ]:
# I will create a sample of only the non_negative
clean_df= taxi_data[taxi_data['tip_amount']>=0]

In [ ]:

corr = clean_df[["tip_amount",'passenger_count',"trip_distance","total_amount"]].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)